In [1]:
# filter some warning messages
import warnings 
warnings.filterwarnings("ignore") 

#libraries
import datetime as dt
import xarray as xr
import fsspec
import s3fs
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# make datasets display nicely
xr.set_options(display_style="html")  

#magic fncts #put static images of your plot embedded in the notebook
%matplotlib inline  
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

In [10]:
def get_geo_data(sat,lyr,idyjl):
    # arguments
    # sat   goes-east,goes-west,himawari
    # lyr   year
    # idyjl day of year
    
    d = dt.datetime(lyr,1,1) + dt.timedelta(days=idyjl)
    fs = s3fs.S3FileSystem(anon=True) #connect to s3 bucket!

    #create strings for the year and julian day
    imon,idym=d.month,d.day
    syr,sjdy,smon,sdym = str(lyr).zfill(4),str(idyjl).zfill(3),str(imon).zfill(2),str(idym).zfill(2)
    print(syr,sjdy,smon,sdym)
    
    #use glob to list all the files in the directory
    if sat=='goes-east':
        file_location,var = fs.glob('s3://noaa-goes16/ABI-L2-SSTF/'+syr+'/'+sjdy+'/*/*.nc'),'SST'
    if sat=='goes-west':
        file_location,var = fs.glob('s3://noaa-goes17/ABI-L2-SSTF/'+syr+'/'+sjdy+'/*/*.nc'),'SST'
    if sat=='himawari':
        file_location,var = fs.glob('s3://noaa-himawari8/AHI-L2-FLDK-SST/'+syr+'/'+smon+'/'+sdym+'/*/*L2P*.nc'),'sea_surface_temperature'
    
    print(file_location)
    
    #make a list of links to the file keys
    if len(file_location)<1:
        return file_ob
    file_ob = [fs.open(file) for file in file_location]        #open connection to files
    
    #open all the day's data
    ds = xr.open_mfdataset(file_ob,combine='nested',concat_dim='time') #note file is super messed up formatting
    
    #clean up coordinates which are a MESS in GOES
    #rename one of the coordinates that doesn't match a dim & should
    if not sat=='himawari':
        ds = ds.rename({'t':'time'})
        ds = ds.reset_coords()
        
        #clean up variables that should be attributes
        varlist = ['retrieval_solar_zenith_angle', 'day_solar_zenith_angle', 'y_image', 'retrieval_local_zenith_angle', 'x_image', 'night_solar_zenith_angle', 'quantitative_local_zenith_angle',
           'SST_night_only_emissive_wavelength', 'SST_night_only_emissive_band_id', 'SST_day_night_emissive_band_ids', 'SST_day_night_emissive_wavelengths']
        for var in varlist:
            try : 
                ds[var]
            except KeyError: 
                pass
            else: 
                ds.attrs[var]=str(ds.variables[var].attrs['long_name']) + ' (' + str(ds.variables[var].attrs['units']) + ') = ' + str(ds.variables[var].values)
                ds = ds.drop(var, dim=None)

    else:
        ds = ds.rename({'ni':'x','nj':'y'})
    
    #put in to Celsius
    #ds[var] -= 273.15   #nice python shortcut to +- from itself a-=273.15 is the same as a=a-273.15
    #ds[var].attrs['units'] = '$^\circ$C'
   
    return ds


In [14]:
%%time
lyr, idyjl = 2020, 210  #may 30, 2020

sat = 'himawari'
ds = get_geo_data(sat,lyr,idyjl)

#print(syr,sjdy,smon,sdym)
ds

2020 210 07 29
['noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0000/20200729000000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0100/20200729010000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0200/20200729020000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0300/20200729030000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0400/20200729040000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0500/20200729050000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0600/20200729060000-STAR-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.71-v02.0-fv01.0.nc', 'noaa-himawari8/AHI-L2-FLDK-SST/2020/07/29/0700/20200729070000-STAR-L2P_GHRSST-SSTsubski

<xarray.Dataset>
Dimensions:                       (time: 24, x: 5500, y: 5500)
Coordinates:
  * x                             (x) float32 -0.15366402 ... 0.15366402
  * y                             (y) float32 0.15366402 ... -0.15366402
  * time                          (time) object 2020-07-29 00:00:00 ... 2020-...
Data variables:
    sst_dtime                     (time, y, x) timedelta64[ns] dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    satellite_zenith_angle        (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    sea_surface_temperature       (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    brightness_temperature_08um6  (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    brightness_temperature_10um4  (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    brightness_temperature_11um2  (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    brightness_temperature_12um3  (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    sses_bias                     (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    sses_standard_deviation       (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    dt_analysis                   (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    wind_speed                    (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    l2p_flags                     (time, y, x) int16 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    quality_level                 (time, y, x) float32 dask.array<chunksize=(1, 5500, 5500), meta=np.ndarray>
    geostationary                 (time) int32 -2147483647 ... -2147483647
Attributes:
    CFAC:                       [20466275]
    COFF:                       [2750.5]
    Coefficient_for_Sd:         [1.73712226e+09]
    Dist_Virt_Sat:              [42164.]
    Earth_Radius_Equator:       [6378.137]
    Earth_Radius_Polar:         [6356.7523]
    LFAC:                       [20466275]
    LOFF:                       [2750.5]
    Nadir_latitude:             [-0.03231456]
    Nadir_longitude:            [140.2600616]
    Req2_info:                  [1.0067395]
    Req_info:                   [0.00669438]
    Rpol_info:                  [0.99330562]
    SSP_latitude:               [-0.00803651]
    SSP_longitude:              [140.66196304]
    Sub_Lon:                    [140.7]
    Sun_Pos:                    [-8.94166471e+07  1.12654081e+08  4.88327422e...
    destripe:                   no
    easternmost_longitude:      [-138.07843]
    northernmost_latitude:      [81.05099]
    southernmost_latitude:      [-81.05099]
    westernmost_longitude:      [59.47843]
    Conventions:                CF-1.7
    Metadata_Conventions:       Unidata Dataset Discovery v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              swath
    comment:                    This file contains sea surface temperature da...
    creator_email:              Alex.Ignatov@noaa.gov
    creator_name:               Alex Ignatov
    creator_url:                http://www.star.nesdis.noaa.gov
    gds_version_id:             02.0
    geospatial_lat_units:       degrees_north
    geospatial_lon_units:       degrees_east
    institution:                NOAA/NESDIS/STAR
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    GHRSST protocol describes data use as free an...
    naming_authority:           org.ghrsst
    netcdf_version_id:          4.6.3 of Apr  2 2019 11:58:26 $
    processing_level:           L2P
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_email:            ghrss

In [47]:
#np.nanmax(ds.percent_uncorrectable_L0_errors.values)
#ds.SST_day_night_emissive_wavelengths.values
#ds['retrieval_solar_zenith_angle']
#ds.SST_day_night_emissive_band_ids.values
#ds = ds.drop('SST_day_night_emissive_band_ids', dim=None)
#ds

In [55]:
varlist = ['retrieval_solar_zenith_angle', 'day_solar_zenith_angle', 'y_image', 'retrieval_local_zenith_angle', 'x_image', 'night_solar_zenith_angle', 'quantitative_local_zenith_angle',
           'SST_night_only_emissive_wavelength', 'SST_night_only_emissive_band_id', 'SST_day_night_emissive_band_ids', 'SST_day_night_emissive_wavelengths']

for var in varlist:
    try : 
        ds[var]
    except KeyError: 
        pass
    else: 
        ds.attrs[var]=str(ds.variables[var].attrs['long_name']) + ' (' + str(ds.variables[var].attrs['units']) + ') = ' + str(ds.variables[var].values)
        ds = ds.drop(var, dim=None)
#ds = ds.drop(dim='SST_day_night_emissive_bands')
ds
#ds.variables['x_image'].values

<xarray.Dataset>
Dimensions:                                                 (number_of_LZA_bounds: 2, number_of_SZA_bounds: 2, number_of_image_bounds: 2, number_of_time_bounds: 2, time: 24, x: 5424, y: 5424)
Coordinates:
  * y                                                       (y) float32 0.151...
  * x                                                       (x) float32 -0.15...
  * time                                                    (time) datetime64[ns] ...
Dimensions without coordinates: number_of_LZA_bounds, number_of_SZA_bounds, number_of_image_bounds, number_of_time_bounds
Data variables:
    SST                                                     (time, y, x) float32 dask.array<chunksize=(1, 5424, 5424), meta=np.ndarray>
    DQF                                                     (time, y, x) float32 dask.array<chunksize=(1, 5424, 5424), meta=np.ndarray>
    time_bounds                                             (time, number_of_time_bounds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    goes_imager_projection                                  (time) int32 -214...
    y_image_bounds                                          (time, number_of_image_bounds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    x_image_bounds                                          (time, number_of_image_bounds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    nominal_satellite_subpoint_lat                          (time) float64 0....
    nominal_satellite_subpoint_lon                          (time) float64 -1...
    nominal_satellite_height                                (time) float64 3....
    geospatial_lat_lon_extent                               (time) float32 9....
    total_number_of_good_quality_ocean_pixels               (time) float64 6....
    total_number_of_degraded_quality_ocean_pixels           (time) float64 9....
    total_number_of_severely_degraded_quality_ocean_pixels  (time) float64 1....
    total_number_of_unprocessed_pixels                      (time) float64 8....
    number_of_day_SST_pixels                                (time) float64 5....
    number_of_night_SST_pixels                              (time) float64 6....
    number_of_twilight_SST_pixels                           (time) float64 1....
    sea_surface_temp_outlier_pixel_count                    (time) float64 2....
    minimum_sea_surface_temp                                (time) float64 18...
    maximum_sea_surface_temp                                (time) float64 32...
    mean_sea_surface_temp                                   (time) float64 28...
    standard_deviation_sea_surface_temp                     (time) float64 16...
    min_obs_modeled_diff_SST_night_only_emissive_band       (time) float64 -3...
    max_obs_modeled_diff_SST_night_only_emissive_band       (time) float64 1....
    mean_obs_modeled_diff_SST_night_only_emissive_band      (time) float64 -0...
    std_dev_obs_modeled_diff_SST_night_only_emissive_band   (time) float64 0....
    min_retrieved_Reynolds_SST_diff                         (time) float64 -1...
    max_retrieved_Reynolds_SST_diff                         (time) float64 21...
    mean_retrieved_Reynolds_SST_diff                        (time) float64 0....
    std_dev_retrieved_Reynolds_SST_diff                     (time) float64 0....
    algorithm_dynamic_input_data_container                  (time) int32 -214...
    processing_parm_version_container                       (time) int32 -214...
    algorithm_product_version_container                     (time) int32 -214...
    retrieval_local_zenith_angle_bounds                     (time, number_of_LZA_bounds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    quantitative_local_zenith_angle_bounds                  (time, number_of_LZA_bounds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    twilight_solar_zenith_angle                             (time) float32 85...
    retrieval_solar_zenith_angle_bounds     

In [ ]:
%%time
subset = ds.sel(x=slice(-0.01,0.07215601),y=slice(0.12,0.09))  #reduce to GS region

masked = subset.SST.where(subset.DQF==0)

masked.isel(time=14).plot(vmin=14+273.15,vmax=30+273.15,cmap='inferno')

In [ ]:
#%%time

#mean_dy = masked.mean('time',skipna=True)   #here I want all possible values so skipna=True

#mean_dy.plot(vmin=14+273.15,vmax=30+273.15,cmap='inferno')